In [18]:
from crop_features import crop_features

In [19]:
annotation_file = './datasets/t1_train/t1_train_coco.json'
image_dir = './data/coco/images'
objects_arr, class_arr, img_id_arr, bbox_arr = crop_features(annotation_file, image_dir, FH = 384, FW = 600)

Image set created with shape  torch.Size([100, 3, 384, 600])
Features created with shape torch.Size([100, 256, 96, 150])
Number of objects obtained :  665


In [20]:
#copy of kmeans.py #updated

import torch
import time
from pykeops.torch import LazyTensor

use_cuda = torch.cuda.is_available()
dtype = torch.float32 if use_cuda else torch.float64
device_id = "cuda:0" if use_cuda else "cpu"


def KMeans(x, K=10, Niter=10, verbose=True):
    """Implements Lloyd's algorithm for the Euclidean metric."""

    start = time.time()
    N, D = x.shape  # Number of samples, dimension of the ambient space

    c = x[:K, :].clone()  # Simplistic initialization for the centroids

    x_i = LazyTensor(x.view(N, 1, D))  # (N, 1, D) samples
    c_j = LazyTensor(c.view(1, K, D))  # (1, K, D) centroids

    # K-means loop:
    # - x  is the (N, D) point cloud,
    # - cl is the (N,) vector of class labels
    # - c  is the (K, D) cloud of cluster centroids
    for i in range(Niter):

        # E step: assign points to the closest cluster -------------------------
        D_ij = ((x_i - c_j) ** 2).sum(-1)  # (N, K) symbolic squared distances
        cl = D_ij.argmin(dim=1).long().view(-1)  # Points -> Nearest cluster

        # M step: update the centroids to the normalized cluster average: ------
        # Compute the sum of points per cluster:
        c.zero_()
        c.scatter_add_(0, cl[:, None].repeat(1, D), x)

        # Divide by the number of points per cluster:
        Ncl = torch.bincount(cl, minlength=K).type_as(c).view(K, 1)
        c /= Ncl  # in-place division to compute the average

    if verbose:  # Fancy display -----------------------------------------------
        if use_cuda:
            torch.cuda.synchronize()
        end = time.time()
        print(
            f"K-means for the Euclidean metric with {N:,} points in dimension {D:,}, K = {K:,}:"
        )
        print(
            "Timing for {} iterations: {:.5f}s = {} x {:.5f}s\n".format(
                Niter, end - start, Niter, (end - start) / Niter
            )
        )

    return cl, c

In [21]:
#copy of cluster.py part 1 #updated

import numpy as np
import pandas as pd
from statistics import mode

#get the class name of a given object class id
def get_class_name(class_id, annotation_file):
    
    import json
    with open(annotation_file, 'r') as file:
        _data = json.load(file)
        _classes = _data["categories"]

        for category in _classes:
            if(category["id"] == class_id):
                return category["name"]

#get a dictionary with all object class ids and their names
def get_all_class_names(class_arr, annotation_file):

    classes = set(class_arr)
    all_class_names = {}

    for i in list(classes):
        cls_name = get_class_name(i, annotation_file)
        all_class_names[i] = cls_name
    return all_class_names


#get cluster names for the predicted kmeans cluster labels
def get_cluster_names(class_arr, label, all_class_names):
    clusters = {}
    for i in range(len(label)):
        cluster_id = label[i]
        cluster_name = all_class_names[class_arr[i]]
        
        if cluster_id not in clusters.keys():
            clusters[cluster_id] = []
            pass
        clusters[cluster_id].append(cluster_name)

    cluster_names = {}
    for i in clusters.keys():
        cluster_names[i] = mode(clusters[i])
    return cluster_names

In [22]:
#update for cluster.py part 2 (pred_clusters functions)
data = np.array([x.numpy() for x in objects_arr], dtype=np.float32)
x = torch.from_numpy(data)
cl, c = KMeans(x, K=10, Niter=10, verbose=True)

K-means for the Euclidean metric with 665 points in dimension 256, K = 10:
Timing for 10 iterations: 0.01546s = 10 x 0.00155s



In [23]:
c.shape

torch.Size([10, 256])

In [24]:
cl.shape

torch.Size([665])